In [1]:
import itertools
import collections

def read_input(day):
    """Read the input file for the day"""
    filename = 'input/input{}.txt'.format(day)
    with open(filename) as f:
        yield from f.read().splitlines()

# Day 1: Chronal Calibration

## Part 1


> For example, if the device displays frequency changes of +1, -2, +3, +1, then starting from a frequency of zero, the following changes would occur:

> Current frequency  0, change of +1; resulting frequency  1.
> Current frequency  1, change of -2; resulting frequency -1.
> Current frequency -1, change of +3; resulting frequency  2.
> Current frequency  2, change of +1; resulting frequency  3.
> In this example, the resulting frequency is 3.


Straightforward, just sum all the numbers in the input file.


In [2]:
sum(int(x) for x in read_input(1))

497

## Part 2

> You notice that the device repeats the same frequency change list over and over. To calibrate the device, you need to find the first frequency it reaches twice.

Now we need to loop through the input list and keep track of already seen frequencies.

In [3]:
def first_duplicate(L):
    f = 0
    seen = set()
    for fc in itertools.cycle(L):
        f += int(fc)
        if f in seen:
            return f
        else:
            seen.add(f)

first_duplicate(read_input(1))

558

# Day 2: Inventory Management System

## Part 1


> To make sure you didn't miss any, you scan the likely candidate boxes again, counting the number that have an ID containing exactly two of any letter and then separately counting those with exactly three of any letter. You can multiply those two counts together to get a rudimentary checksum and compare it to what your device predicts.

OK, so we first need to determine the frequency of characters in the inventory ID. `collections.Counter` to the rescue!

The `magic_frequency` function will, for a given string, return the resunt of the `[2x, 3x]` character test.
Afterwards, we need to sum these results. `zip()` can be used to transform the `[[2x_1, 3x_1], .. [2x_n, 3x_n]]` list to `[[2x_1, .. 2x_n], [3x_1, .. 3x_n]]`.

In [4]:
def magic_frequency(s):
    """Determine if the given string contains 2x letter or 3x letter, return list"""
    freq = collections.Counter(s)
    c = [0,0]
    if 2 in freq.values():
        c[0] = 1
    if 3 in freq.values():
        c[1] = 1
    return c

assert(magic_frequency('abcdef') == [0,0])
assert(magic_frequency('bababc') == [1,1])

checksum = list(zip(*[magic_frequency(s) for s in read_input(2)]))
sum(checksum[0]) * sum(checksum[1])

5456

## Part 2
Now we need to find two inventory IDs that differ by exactly one character. In other words, need to find two strings with edit distance of 1. First, let's sort the inputs, then compare only the neighbors

In [5]:
def edit_distance(s1, s2):
    m=len(s1)+1
    n=len(s2)+1

    tbl = {}
    for i in range(m): tbl[i,0]=i
    for j in range(n): tbl[0,j]=j
    for i in range(1, m):
        for j in range(1, n):
            cost = 0 if s1[i-1] == s2[j-1] else 1
            tbl[i,j] = min(tbl[i, j-1]+1, tbl[i-1, j]+1, tbl[i-1, j-1]+cost)

    return tbl[i,j]

def find_inventory(L):
    L.sort()
    for idx, t in enumerate(L):
        if idx == 0:
            continue
        if edit_distance(t, L[idx-1]) == 1:
            same = ''
            for c_idx, c in enumerate(t):
                if c == L[idx-1][c_idx]:
                    same += c
            return same

find_inventory(list(read_input(2)))

'megsdlpulxvinkatfoyzxcbvq'